In [38]:
import os
import ROOT
##### SELECTING GENERATOR FOR SIGNAL SAMPLE
signal_sample="Ztautau_MG.root"
post_fit = False
if not post_fit:
    fit_scale = 1.0
    type_of_mc = '_PreFit'
else :
    fit_scale = 1.074
    type_of_mc = '_PosFit'

#### CREATING THE CORRESPONDING MC.ROOT FILE EVERYTIME YOU CHANGE THE SIGNAL SAMPLE

os.system("rm BG.root MC.root")
os.system('hadd BG.root VV.root singletop.root Wjets.root ttbar.root Zjet.root')
os.system("hadd MC.root "+"BG.root "+signal_sample)

0

hadd Target file: BG.root
hadd compression setting for all output: 1
hadd Source file 1: VV.root
hadd Source file 2: singletop.root
hadd Source file 3: Wjets.root
hadd Source file 4: ttbar.root
hadd Source file 5: Zjet.root
hadd Target path: BG.root:/
hadd Target file: MC.root
hadd compression setting for all output: 1
hadd Source file 1: BG.root
hadd Source file 2: Ztautau_MG.root
hadd Target path: MC.root:/


In [39]:
#### HISTOGRAMS FOR PLOTTING

histos_no_rebin={
"lepiso":[],
}
histos_rebin={
"delta_phi":[[2.0],[0.2,0.8],0.2],
"reco_mass":[[70,110,140],[10,5,10,20],5],
}
    
histos={}
histos.update(histos_no_rebin)
histos.update(histos_rebin)

In [40]:
Data = {"Data":["Data.root",ROOT.kBlack,0]}   
Signal = {"Signal":[signal_sample,ROOT.kOrange+7,0]} 
Background = {"Higgs":["Higgs.root",ROOT.kRed,0],"Zjets":["Zjet.root",ROOT.kViolet,0],"Wjets":["Wjets.root",ROOT.kGreen,0],"ttbar":["ttbar.root",ROOT.kYellow,0],"VV":["VV.root",ROOT.kBlue,0],"SingleTop":["singletop.root",ROOT.kCyan,0]}


%run "../../Scripts/Plotter.ipynb"

Plot(Data,Signal,Background,histos,type_of_mc,fit_scale,after_fit=post_fit,final_state="Z#rightarrow #tau#tau")

lepiso
delta_phi
0.0
3.2
[0.  0.2 0.4 0.6 0.8 1.  1.2 1.4 1.6 1.8 2.  3.2]
reco_mass
0.0
240.0
[  0.  10.  20.  30.  40.  50.  60.  70.  75.  80.  85.  90.  95. 100.
 105. 110. 120. 130. 140. 160. 180. 200. 220. 240.]


Info in <TCanvas::Print>: pdf file lepiso__PreFit.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas2
Info in <TCanvas::Print>: pdf file delta_phi__PreFit.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas2
Info in <TCanvas::Print>: pdf file reco_mass__PreFit.pdf has been created
